## Mastering Machine Learning 2025

Taller 3: clasificación de texto usando modelos de lenguaje

Antes de iniciar abra este cuaderno en Google Colab y habilite la ejecución con GPU:
- En el menú Entorno de ejecución seleccione Cambiar tipo entorno de ejecución.
- Asegúrese de tener seleccionado Python 3.
- Como Acelerador de hardware seleccione GPU T4.

Instale las dependencias para asegurar la correcta ejecución del cuaderno.

In [ ]:
!pip install --upgrade datasets huggingface_hub

Usaremos el dataset de reseñas de pelítuclas de Rotten Tomatoes, disponible en Hugging Face

In [ ]:
from datasets import load_dataset

# Load our data
data = load_dataset('cornell-movie-review-data/rotten_tomatoes')


Exploremos los datos, que contienen conjuntos de entrenamiento, validación y prueba

In [ ]:
data

Exploremos algunos registros, que contienen tanto el texto de la reseña, como la etiqueta, positiva o negativa, de acuerdo con el sentimiento de la reseña.

In [ ]:
data["train"][1]

In [ ]:
data["train"][5000]

### Clasificación usando un modelo entrenado para una tarea específica

Ahora vamos a clasificar las reseñas de películas usando un modelo pre-entrenado para la **tarea específica** de **análisis de sentimiento**. Específicamente, usaremos el modelos RoBERTa (https://huggingface.co/FacebookAI/roberta-base) que es un modelo BERT entrenado de manera auto-supervisada empleando tweets en inglés.



In [ ]:
from transformers import pipeline

# Path del modelo
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# Cargar el modelo en un pipeline para ejecutar primero el tokenizador
# (mismo del modelo) y luego el modelo
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred = []
# Aplicamos el pipeline a cada elemento del dataset y obtenemos un output
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
  # con base en el output calculamos el puntaje positivo y negativo
  negative_score = output[0]["score"]
  positive_score = output[2]["score"]
  # asignamos la clase positiva o negativa dependiendo de los puntajes obtenidos
  assignment = np.argmax([negative_score, positive_score])
  y_pred.append(assignment)

Definimos una función que genera el reporte de clasificación con las métricas más importantes

In [ ]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

Usamos la función con las predicciones obtenidas

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

### Clasificación usando un modelo de embeddings

Ahora vamos a clasificar las reseñas de películas usando un modelo de embeddings que no ha sido pre-entrenado para alguna tarea. Específicamente, usaremos el paquete sentence_transformers y el modelo all-mpnet-base-v2 (https://huggingface.co/sentence-transformers/all-mpnet-base-v2) que mapea frases y párrafos a un espacio de 768 dimensiones, una representación que puede usarse para tareas de clasificación, clustering, etc.

In [ ]:
from sentence_transformers import SentenceTransformer

# Cargamos el modelo
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


# Convertimos los testos de las reseñas a embeddings
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

Si revisamos el tamaño de los datos obtenidos podemodes observar que cada texto ha sido transformado a un vector de 768 dimensiones.

In [ ]:
train_embeddings.shape

Ahora usaremos esta representación numérica de los textos y las etiquetas para entrenar un modelo de clasificación (regresión logística). Note que usamos los datos de entrenamiento para esto.

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

Ahora evaluamos el desempeño del modelo de clasificación usando los datos de prueba e imprimimos el informe con las métricas.

In [ ]:
y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

### Clasificación sin etiquetas - Zero-shot classification

Para evaluar el desempeño de los modelos anteriores usamos etiquetas para indicar si las reseñas eran positivas o negativas. Esta tarea típicamente es muy costosa y puede hacer infactible un proyecto. En esta sección exploramos una alternativa empleando modelos preentrenados de embeddings.

Usaremos el mismo modelo de embeddings usado anterioremente, el modelo all-mpnet-base-v2 (https://huggingface.co/sentence-transformers/all-mpnet-base-v2) que mapea frases y párrafos a un espacio de 768 dimensiones, una representación que puede usarse para tareas de clasificación, clustering, etc.

El primer paso es encontrar una representación de las **etiquetas de las clases** en el espacio del embedding. En este caso las etiquetas se refieren a reseñas positivas y negativas, así que usamos un texto para cada clase que las represente.

In [ ]:
label_embeddings = model.encode(["A negative review",  "A positive review"])

Ahora importamos la función de cosine_similarity para calcular la similitud de coseno entre un par de vectores. Si los vectores son iguales (apuntan en la misma dirección), el ángulo entre ellos es 0 y su coseno es 1. Por el contrario, si los vectores son diferentes, el ángulo es diferente a cero y el coseno es menor a 1, indicando una menor similitud.

Calculamos entonces la matriz de similtud entre los embeddings de los casos de prueba y los embeddings de las etiquetas que definimos arriba. Y hacemos las predicciones seleccionando la clase que presenta la mayor similitud para cada caso.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

Evaluamos el desempeño usando el mismo reporte.

In [ ]:

evaluate_performance(data["test"]["label"], y_pred)